In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler

# Standard sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(y):  # y is output
    return y * (1 - y)

# Bipolar sigmoid
def bipolar_sigmoid(x):
    return (2 / (1 + np.exp(-x))) - 1

def bipolar_sigmoid_derivative(y):  # y is output
    return 0.5 * (1 - y ** 2)

# MLP class (custom activation)
class MLP:
    def __init__(self, input_size, hidden_size, activation="sigmoid", lr=0.1, epochs=5000):
        self.lr = lr
        self.epochs = epochs
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.w1 = np.random.uniform(-1, 1, (input_size, hidden_size))
        self.b1 = np.random.uniform(-1, 1, (1, hidden_size))
        self.w2 = np.random.uniform(-1, 1, (hidden_size, 1))
        self.b2 = np.random.uniform(-1, 1, (1, 1))

        if activation == "sigmoid":
            self.act = sigmoid
            self.act_deriv = sigmoid_derivative
        elif activation == "bipolar":
            self.act = bipolar_sigmoid
            self.act_deriv = bipolar_sigmoid_derivative
        else:
            raise ValueError("Unsupported activation function")

    def train(self, X, y):
        for _ in range(self.epochs):
            h_input = np.dot(X, self.w1) + self.b1
            h_output = self.act(h_input)

            o_input = np.dot(h_output, self.w2) + self.b2
            o_output = self.act(o_input)

            error = y - o_output
            d_output = error * self.act_deriv(o_output)

            error_hidden = d_output.dot(self.w2.T)
            d_hidden = error_hidden * self.act_deriv(h_output)

            # Update weights and biases
            self.w2 += self.lr * h_output.T.dot(d_output)
            self.b2 += self.lr * np.sum(d_output, axis=0, keepdims=True)
            self.w1 += self.lr * X.T.dot(d_hidden)
            self.b1 += self.lr * np.sum(d_hidden, axis=0, keepdims=True)

    def predict(self, X):
        h = self.act(np.dot(X, self.w1) + self.b1)
        o = self.act(np.dot(h, self.w2) + self.b2)
        return np.round(o), o  # predicted class, raw output

# ------------------------- Main Program -------------------------

# Generate linearly separable data
X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_classes=2, random_state=1)

# Scale features to [-1, 1] for bipolar compatibility
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

# Reshape targets for MLP
y_sigmoid = y.reshape(-1, 1)  # 0 or 1
y_bipolar = (2 * y - 1).reshape(-1, 1)  # -1 or +1

# Train with standard sigmoid activation
mlp_sigmoid = MLP(input_size=2, hidden_size=4, activation="sigmoid")
mlp_sigmoid.train(X_scaled, y_sigmoid)
pred_sigmoid, _ = mlp_sigmoid.predict(X_scaled)
acc_sigmoid = np.mean(pred_sigmoid == y_sigmoid)

# Train with bipolar sigmoid activation
mlp_bipolar = MLP(input_size=2, hidden_size=4, activation="bipolar")
mlp_bipolar.train(X_scaled, y_bipolar)
pred_bipolar, _ = mlp_bipolar.predict(X_scaled)
acc_bipolar = np.mean(pred_bipolar == y_bipolar)

# ------------------------- Results -------------------------
print(f"Accuracy with standard sigmoid: {acc_sigmoid * 100:.2f}%")
print(f"Accuracy with bipolar sigmoid:  {acc_bipolar * 100:.2f}%")


Accuracy with standard sigmoid: 93.00%
Accuracy with bipolar sigmoid:  80.50%
